# Data Privacy Final Project
### Sam Clark & Josh Childs

For this project, we've decided to compare the accuracy of several normal Convolutional Neural Networks to their counter parts that will use differential privacy. We will be using the MNIST dataset with the tensflow library.  

In [12]:
import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds
from ipywidgets import IntProgress
from sklearn.metrics import classification_report

tf.enable_v2_behavior()

## Load MNIST Data

In [13]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

## Preprocess Data

In [14]:
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
input_shape = (28, 28, 1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

## Build Models

In [17]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(28, kernel_size=(3,3), input_shape=input_shape),
  tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation=tf.nn.relu),  
  tf.keras.layers.Dropout(0.3),
  tf.keras.layers.Dense(10,activation=tf.nn.softmax)
])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 26, 26, 28)        280       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 28)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 4732)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               605824    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 10)                1290      
Total params: 607,394
Trainable params: 607,394
Non-trainable params: 0
________________________________________________

## Train & Save

In [19]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

model.fit(x=x_train,y=y_train, epochs=1, callbacks = [callback])
model.save("models/original")

1875/1875 [==============================] - 19s 10ms/step - loss: 0.5172 - accuracy: 0.8522
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: models/original/assets


## Evaluate Models

In [21]:
model.evaluate(x_test, y_test)

313/313 [==============================] - 1s 3ms/step - loss: 0.2238 - accuracy: 0.9354


[0.22379177808761597, 0.9354000091552734]

## Add Differential Privacy

In [8]:
def tf_gaussian_mech(v, sensitivity, epsilon, delta):
    return v + tf.random.normal(v.shape, mean=0.0, stddev=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)

def tf_l2_clip(v, b):
    norm = tf.norm(v)
    return tf.cond(norm > b, lambda: b * (v / norm), lambda: v)